# Welche Entwickler haben die meisten Commits?

## 1. Verbindung zur Datenbank
Es wird eine Verbindung zur Neo4j-Datenbank aufgebaut.

In [ ]:
import py2neo

graph = py2neo.Graph(bolt=True, host='localhost', user='neo4j', password='neo4j')

## 2. Cypher-Abfrage
Es wird eine Abfrage an die Datenbank gestellt. Das Ergebnis wird in einem Dataframe (pandas) gespeichert.

In [ ]:
import pandas as pd

query ="MATCH (a:Author)-[:COMMITTED]->(c:Commit)-[:CONTAINS_CHANGE]->(:Change)-[:MODIFIES]->(file:File) WHERE NOT c:Merge RETURN a.name as developer, count(distinct c) as commits"
df = pd.DataFrame(graph.run(query).data())


## 3. Datenaufbereitung
Zur Kontrolle werden die ersten fünf Zeilen des Ergebnisses der Abfrage als Tabelle ausgegeben.

In [ ]:
df.head()

Im folgenden Codeabschnitt wird das Dataframe absteigend entsprechend der Anzahl der Commits mit der Methode ```sort_values``` sortiert und die ersten 10 Zeilen im Dataframe ```developer_df``` abgebildet (https://www.data-science-architect.de/selektieren-von-daten-in-dataframes/).

In [ ]:
# Sortiere das Dataframe nach der Anzahl der Commits (commits).
df = df.sort_values('commits', ascending=False)

# Extrahiere die ersten zehn Spalten und speichere sie im Dataframe developer_df.
developer_df = df[0:10]
developer_df.head(10)


# 4. Visualisierung
Die Daten werden mittels eines Bar Charts visualisiert (https://plot.ly/python/horizontal-bar-charts/, x=x_commits, y=y_developer).

In [ ]:
from IPython.display import display, HTML

base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
  <script type="text/javascript" src="https://kozea.github.io/pygal.js/2.0.x/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""

In [ ]:
# Erstelle Bar Chart.
import pygal
bar_chart = pygal.HorizontalBar(show_legend=True, human_readable=True, 
fill=True, legend_at_bottom=True, legend_at_bottom_columns=2)
bar_chart.title = 'Entwickler mit den meißten Commits'
for index, row in developer_df.iterrows():
     bar_chart.add(row['developer'],[{"value": row['commits']}])
display(HTML(base_html.format(rendered_chart=bar_chart.render(is_unicode=True))))
